<a href="https://colab.research.google.com/github/Donggeon2960/623_chungnam/blob/main/709_opencv_basic_%EA%B8%B0%EB%B3%B8%EC%A0%81%EC%9D%B8_MOG2_%EC%B0%A8%EB%9F%89_%EA%B0%90%EC%A7%80_%EC%BD%94%EB%93%9C_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

opencv 기본 MOG2 차량 감지 코드

In [ ]:
# 1. 기본 MOG2 차량 감지 코드
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:   # 30초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()  # cap.release()는 꼭 해줘야 합니다! cap.release()를 하는 이유: release() 안 하면 → 파일이 "사용 중" 상태로 남음 메모리 문제 발생
print("처리 완료!")

# 다음 방법으로 위의 코드를 수정해주세요  
try:
    cap = cv2.VideoCapture('video.mp4')
    # ... 처리 ...
finally:
    cap.release()  # 항상 실행됨

### 2. 기본 MOG2 차량 감지 코드 +noise
### 노이즈 제거 기능
추가된 내용:

Opening 연산: 작은 노이즈(점들) 제거
Closing 연산: 차량 내부의 작은 구멍들 메우기
3x3 타원형 커널: 자연스러운 형태로 처리

효과:

배경의 작은 노이즈점들이 사라짐
차량의 형태가 더 깔끔하게 표시됨
차량 내부의 빈 공간들이 채워짐

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:  # 30초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("처리 완료!")

바운딩박스 추가  결과가 차량이 아닌것도 인식 이게 한계

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 3000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:
        # 결과를 나란히 표시 (바운딩 박스가 있는 프레임 + 마스크)
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("처리 완료!")

종횡비 필터 추가 - 차량 같은 비율만 인식

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 2000:
            x, y, w, h = cv2.boundingRect(contour)

            # 종횡비 필터 추가 (차량은 보통 가로가 세로보다 길거나 비슷함)
            aspect_ratio = w / h
            if 0.5 <= aspect_ratio <= 4.0:  # 너무 세로로 길거나 가로로 긴 것 제외
                cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:
        # 결과를 나란히 표시 (바운딩 박스가 있는 프레임 + 마스크)
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("처리 완료!")

ROI 기능 추가완료!  참량감지가 목
추가된 기능:

create_roi_mask() 함수: 도로 영역만 선택하는 마스크 생성
ROI 영역: 화면 아래쪽 절반 (실제 차량이 다니는 도로 부분)
cv2.bitwise_and(): ROI 영역 밖은 분석에서 제외

효과:

하늘, 건물, 나무 등 위쪽 배경은 완전히 제외
도로 영역의 차량만 감지
오탐지 대폭 감소

ROI 조정 방법:

height//2 부분을 height//3 으로 바꾸면 더 작은 영역
roi_points 좌표를 수정하면 다른 모양의 ROI 가능

이제 도로 영역의 차량만 정확하게 감지될 겁니다!

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

# ROI 설정 (도로 영역만 분석) - 좌표는 영상에 맞게 조정 필요
def create_roi_mask(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 도로 영역을 다각형으로 설정 (예시 - 실제 영상에 맞게 조정)
    roi_points = np.array([
        [0, height//2],           # 왼쪽 중간
        [width, height//2],       # 오른쪽 중간
        [width, height],          # 오른쪽 아래
        [0, height]               # 왼쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # ROI 마스크 생성 및 적용
    roi_mask = create_roi_mask(frame)
    fgMask = cv2.bitwise_and(fgMask, roi_mask)  # ROI 영역만 남김

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 2000:
            x, y, w, h = cv2.boundingRect(contour)

            # 종횡비 필터 추가 (차량은 보통 가로가 세로보다 길거나 비슷함)
            aspect_ratio = w / h
            if 0.5 <= aspect_ratio <= 4.0:  # 너무 세로로 길거나 가로로 긴 것 제외
                cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 10 == 0:
        # 결과를 나란히 표시 (바운딩 박스가 있는 프레임 + 마스크)
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("처리 완료!")

###차량카운팅   배경 차분법의 한계  딥러닝이 필요한 이유 느낌.전처리 단계로만 사
차량 카운팅 기능 추가 완료!
추가된 기능:

### 파란색 카운팅 라인: 화면 하단 2/3 지점에 가로선
차량 카운트: 바운딩 박스 중심이 라인을 통과하면 카운트
중복 방지: 같은 차량을 여러 번 세지 않도록 ID 관리
실시간 표시:

### 화면 좌상단에 "Vehicle Count: X" 표시
콘솔에 "차량 감지! 총 X대" 출력
각 차량에 "Vehicle" 라벨 표시



### 카운팅 원리:

차량의 바운딩 박스 중심점 계산
중심점이 파란색 라인 근처(±20픽셀)에 있는지 확인
새로운 차량이면 카운트 증가

이제 차량이 파란색 라인을 지날 때마다 자동으로 카운팅됩니다!###

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()

# 차량 카운팅을 위한 변수들
vehicle_count = 0
counted_vehicles = set()  # 중복 카운팅 방지
line_y = None  # 카운팅 라인 위치 (첫 프레임에서 설정)

# ROI 설정 (도로 영역만 분석) - 좌표는 영상에 맞게 조정 필요
def create_roi_mask(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 도로 영역을 다각형으로 설정 (예시 - 실제 영상에 맞게 조정)
    roi_points = np.array([
        [0, height//2],           # 왼쪽 중간
        [width, height//2],       # 오른쪽 중간
        [width, height],          # 오른쪽 아래
        [0, height]               # 왼쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # ROI 마스크 생성 및 적용
    roi_mask = create_roi_mask(frame)
    fgMask = cv2.bitwise_and(fgMask, roi_mask)  # ROI 영역만 남김

    # 노이즈 제거 (모폴로지 연산)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    # 윤곽선 검출 및 바운딩 박스
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    # 카운팅 라인 설정 (첫 프레임에서)
    if line_y is None:
        line_y = frame.shape[0] * 2 // 3  # 화면 아래쪽 2/3 지점

    # 카운팅 라인 그리기
    cv2.line(result_frame, (0, line_y), (frame.shape[1], line_y), (255, 0, 0), 3)

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 2000:
            x, y, w, h = cv2.boundingRect(contour)

            # 종횡비 필터 추가 (차량은 보통 가로가 세로보다 길거나 비슷함)
            aspect_ratio = w / h
            if 0.5 <= aspect_ratio <= 4.0:  # 너무 세로로 길거나 가로로 긴 것 제외
                cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                # 차량 카운팅 (바운딩 박스 중심이 카운팅 라인을 통과하는지 확인)
                center_x = x + w // 2
                center_y = y + h // 2

                # 차량이 카운팅 라인 근처에 있는지 확인
                if abs(center_y - line_y) < 20:  # 라인 근처 20픽셀 이내
                    vehicle_id = f"{center_x}_{frame_count//5}"  # 간단한 ID 생성

                    if vehicle_id not in counted_vehicles:
                        counted_vehicles.add(vehicle_id)
                        vehicle_count += 1
                        print(f"차량 감지! 총 {vehicle_count}대")

                # 바운딩 박스에 차량 번호 표시
                cv2.putText(result_frame, f"Vehicle", (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:
        # 차량 카운트 정보 표시
        cv2.putText(result_frame, f"Vehicle Count: {vehicle_count}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # 결과를 나란히 표시 (바운딩 박스가 있는 프레임 + 마스크)
        combined = np.hstack((result_frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("처리 완료!")

기본 차선 검출 코드입니다!
주요 기능:

그레이스케일 변환: 색상 정보 제거, 에지 검출에 집중
가우시안 블러: 노이즈 제거로 깔끔한 에지 검출
Canny 에지 검출: 차선의 경계선 찾기
사다리꼴 ROI: 차선이 있는 도로 영역만 분석

ROI 모양:

사다리꼴 형태로 원근감 고려
차선이 실제로 있는 영역에 집중
하늘, 건물 등 불필요한 부분 제거

결과 확인:

왼쪽: 원본 영상
오른쪽: 에지 검출 결과 (차선이 흰색 선으로 표시)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 사다리꼴 모양의 ROI (차선이 있는 도로 영역)
    roi_points = np.array([
        [width//4, height],        # 왼쪽 아래
        [width*3//8, height//2],   # 왼쪽 위
        [width*5//8, height//2],   # 오른쪽 위
        [width*3//4, height]       # 오른쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 50, 150)

    # ROI 적용
    roi_mask = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # 결과 프레임 복사
    result_frame = frame.copy()

    # 매 30프레임마다 결과 출력
    if frame_count % 10 == 0:
        # 결과를 나란히 표시 (원본 + 에지)
        combined = np.hstack((result_frame, cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("차선 검출 처리 완료!")

ROI 시각화:

파란색 사다리꼴 선으로 ROI 영역 표시
실제 분석되는 영역을 명확히 확인 가능


영어 라벨:

왼쪽: "Original + ROI" - 원본 영상 + ROI 영역 표시
오른쪽: "Canny Edge Detection" - 에지 검출 결과



결과 확인:

왼쪽 영상에서 파란색 사다리꼴이 ROI 영역
오른쪽에서 사다리꼴 안의 차선들만 에지로 검출
각 화면에 무엇인지 영어로 표시

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 사다리꼴 모양의 ROI (차선이 있는 도로 영역)
    roi_points = np.array([
        [width//4, height],        # 왼쪽 아래
        [width*3//8, height//2],   # 왼쪽 위
        [width*5//8, height//2],   # 오른쪽 위
        [width*3//4, height]       # 오른쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 50, 150)

    # ROI 적용
    roi_mask = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # 결과 프레임 복사
    result_frame = frame.copy()

    # ROI 영역을 시각화 (사다리꼴 모양 표시)
    roi_points = np.array([
        [frame.shape[1]//4, frame.shape[0]],        # 왼쪽 아래
        [frame.shape[1]*3//8, frame.shape[0]//2],   # 왼쪽 위
        [frame.shape[1]*5//8, frame.shape[0]//2],   # 오른쪽 위
        [frame.shape[1]*3//4, frame.shape[0]]       # 오른쪽 아래
    ], np.int32)
    cv2.polylines(result_frame, [roi_points], True, (255, 0, 0), 2)  # 파란색 사다리꼴

    # 매 30프레임마다 결과 출력
    if frame_count % 30 == 0:
        # 영어 라벨 추가
        cv2.putText(result_frame, "Original + ROI", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        edge_display = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)
        cv2.putText(edge_display, "Canny Edge Detection", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 결과를 나란히 표시 (원본+ROI + 에지검출)
        combined = np.hstack((result_frame, edge_display))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("차선 검출 처리 완료!")

차선이 보이게 hHough 변환
HoughLinesP: 확률적 Hough 변환으로 직선 검출
차선 그리기: 검출된 직선을 빨간색으로 표시
3화면 표시: 원본+ROI / 에지검출 / 차선결과

Hough 변환 파라미터:

threshold=50: 직선으로 인정할 최소 점 개수
minLineLength=50: 최소 직선 길이
maxLineGap=50: 직선 간 최대 간격

결과 화면:

왼쪽: 원본 + ROI (파란색 사다리꼴)
가운데: Canny 에지 검출 결과
오른쪽: Lane Detection Result (빨간색 차선)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 사다리꼴 모양의 ROI (차선이 있는 도로 영역)
    roi_points = np.array([
        [width//4, height],        # 왼쪽 아래
        [width*3//8, height//2],   # 왼쪽 위
        [width*5//8, height//2],   # 오른쪽 위
        [width*3//4, height]       # 오른쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 50, 150)

    # ROI 적용
    roi_mask = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # Hough 변환으로 직선 검출
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi/180, threshold=50,
                           minLineLength=50, maxLineGap=50)

    # 결과 프레임들 복사
    result_frame = frame.copy()
    lane_frame = frame.copy()

    # 검출된 차선을 그리기
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(lane_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)  # 빨간색 차선

    # ROI 영역을 시각화 (사다리꼴 모양 표시)
    roi_points = np.array([
        [frame.shape[1]//4, frame.shape[0]],        # 왼쪽 아래
        [frame.shape[1]*3//8, frame.shape[0]//2],   # 왼쪽 위
        [frame.shape[1]*5//8, frame.shape[0]//2],   # 오른쪽 위
        [frame.shape[1]*3//4, frame.shape[0]]       # 오른쪽 아래
    ], np.int32)
    cv2.polylines(result_frame, [roi_points], True, (255, 0, 0), 2)  # 파란색 사다리꼴

    # 매 30프레임마다 결과 출력
    if frame_count % 10 == 0:
        # 영어 라벨 추가
        cv2.putText(result_frame, "Original + ROI", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        edge_display = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)
        cv2.putText(edge_display, "Canny Edge Detection", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.putText(lane_frame, "Lane Detection Result", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 3개 화면을 나란히 표시
        combined = np.hstack((result_frame, edge_display, lane_frame))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("차선 검출 처리 완료!")

미션 사다리꼴 roi시각화 부분을 도로에 맞게 수정한다.

녹색 선 (LEFT): 왼쪽 차선
파란색 선 (RIGHT): 오른쪽 차선
이 선들의 의미:

차량이 주행해야 할 차선의 경계를 표시
자율주행에서 차량이 어느 차선을 따라가야 하는지 알려주는 가이드라인
실제로는 차량이 두 차선 사이를 주행해야 함

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요

# 차선 검출을 위한 ROI 설정
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # 사다리꼴 모양의 ROI (차선이 있는 도로 영역)
    roi_points = np.array([
        [width//4, height],        # 왼쪽 아래
        [width*3//8, height//2],   # 왼쪽 위
        [width*5//8, height//2],   # 오른쪽 위
        [width*3//4, height]       # 오른쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러로 노이즈 제거
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny 에지 검출
    edges = cv2.Canny(blur, 50, 150)

    # ROI 적용
    roi_mask = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    # Hough 변환으로 직선 검출
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi/180, threshold=50,
                           minLineLength=50, maxLineGap=50)

    # 결과 프레임들 복사
    result_frame = frame.copy()
    lane_frame = frame.copy()

    # 차선 분류를 위한 리스트
    left_lines = []
    right_lines = []

    # 검출된 차선을 좌/우로 분류
    if lines is not None:
        img_center = frame.shape[1] // 2  # 화면 중앙

        for line in lines:
            x1, y1, x2, y2 = line[0]

            # 기울기 계산 (수직선 제외)
            if x2 - x1 != 0:
                slope = (y2 - y1) / (x2 - x1)

                # 기울기와 위치로 좌/우 차선 분류
                if slope < -0.3 and x1 < img_center:  # 왼쪽 차선 (음의 기울기)
                    left_lines.append([x1, y1, x2, y2])
                elif slope > 0.3 and x1 > img_center:  # 오른쪽 차선 (양의 기울기)
                    right_lines.append([x1, y1, x2, y2])

    # 차선 연결 함수
    def connect_lane_segments(lines, frame_height):
        if not lines:
            return None

        # 모든 점들을 수집
        points = []
        for x1, y1, x2, y2 in lines:
            points.extend([(x1, y1), (x2, y2)])

        if len(points) < 2:
            return None

        # 최소자승법으로 직선 피팅
        points = np.array(points)
        x_coords = points[:, 0]
        y_coords = points[:, 1]

        # 1차 다항식 피팅
        coeffs = np.polyfit(y_coords, x_coords, 1)

        # 화면 상하단에서의 x 좌표 계산
        y_top = frame_height // 2
        y_bottom = frame_height
        x_top = int(coeffs[0] * y_top + coeffs[1])
        x_bottom = int(coeffs[0] * y_bottom + coeffs[1])

        return [x_top, y_top, x_bottom, y_bottom]

    # 좌측/우측 차선 연결
    left_lane = connect_lane_segments(left_lines, frame.shape[0])
    right_lane = connect_lane_segments(right_lines, frame.shape[0])

    # 연결된 차선 그리기
    if left_lane:
        cv2.line(lane_frame, (left_lane[0], left_lane[1]),
                (left_lane[2], left_lane[3]), (0, 255, 0), 5)  # 녹색 왼쪽 차선
        cv2.putText(lane_frame, "LEFT", (left_lane[0]-30, left_lane[1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if right_lane:
        cv2.line(lane_frame, (right_lane[0], right_lane[1]),
                (right_lane[2], right_lane[3]), (255, 0, 0), 5)  # 파란색 오른쪽 차선
        cv2.putText(lane_frame, "RIGHT", (right_lane[0]+10, right_lane[1]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    # ROI 영역을 시각화 (사다리꼴 모양 표시)
    roi_points = np.array([
        [frame.shape[1]//4, frame.shape[0]],        # 왼쪽 아래
        [frame.shape[1]*3//8, frame.shape[0]//2],   # 왼쪽 위
        [frame.shape[1]*5//8, frame.shape[0]//2],   # 오른쪽 위
        [frame.shape[1]*3//4, frame.shape[0]]       # 오른쪽 아래
    ], np.int32)
    cv2.polylines(result_frame, [roi_points], True, (255, 0, 0), 2)  # 파란색 사다리꼴

    # 매 30프레임마다 결과 출력
    if frame_count % 10 == 0:
        # 영어 라벨 추가
        cv2.putText(result_frame, "Original + ROI", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        edge_display = cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)
        cv2.putText(edge_display, "Canny Edge Detection", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        cv2.putText(lane_frame, "Lane Detection Result", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 3개 화면을 나란히 표시
        combined = np.hstack((result_frame, edge_display, lane_frame))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 후 종료 (테스트용)
    if frame_count > 100:
        break

# 자원 해제
cap.release()
print("차선 검출 처리 완료!")